# Agregación y agrupación

Una pieza fundamental de muchas tareas de análisis de datos es el resumen eficiente: calcular agregaciones como "suma", "media", "mediana", "mínimo" y "máximo", en las que un solo número resume aspectos de un conjunto de datos potencialmente grande.
En este capítulo, exploraremos agregaciones en Pandas, desde operaciones simples similares a las que hemos visto en matrices NumPy hasta operaciones más sofisticadas basadas en el concepto de "groupby".

Por conveniencia, usaremos la misma función mágica "mostrar" que usamos en los capítulos anteriores:

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Datos de planetas

Aquí usaremos el conjunto de datos Planets, disponible a través del [paquete Seaborn](http://seaborn.pydata.org/) (ver [Visualización con Seaborn](04.14-Visualización-With-Seaborn.ipynb)).
Proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como *planetas extrasolares*, o *exoplanetas* para abreviar). Se puede descargar con un simple comando de Seaborn:

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Contiene algunos detalles sobre los más de 1.000 planetas extrasolares descubiertos hasta 2014.

## Agregación simple en Pandas

En ["Agregaciones: mínimo, máximo y todo lo intermedio"](02.04-Computation-on-arrays-aggregates.ipynb), exploramos algunas de las agregaciones de datos disponibles para los arreglos NumPy.
Al igual que con una matriz NumPy unidimensional, para una ``Serie`` de Pandas los agregados devuelven un valor único:

In [4]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [5]:
ser.sum()

2.811925491708157

In [6]:
ser.mean()

0.5623850983416314

Para un `DataFrame`, de forma predeterminada los agregados devuelven resultados dentro de cada columna:

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Al especificar el argumento `axis`, puedes agregar dentro de cada fila:

In [9]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Los objetos Pandas `Series` y `DataFrame` incluyen todos los agregados comunes mencionados en [Agregaciones: mínimo, máximo y todo lo demás] (02.04-Computation-on-arrays-aggregates.ipynb); Además, existe un método conveniente, "describir", que calcula varios agregados comunes para cada columna y devuelve el resultado.
Usemos esto en los datos de Planetas, por ahora eliminando filas con valores faltantes:

In [10]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Este método nos ayuda a comprender las propiedades generales de un conjunto de datos.
Por ejemplo, vemos en la columna "año" que, aunque los exoplanetas se descubrieron ya en 1989, la mitad de todos los planetas en el conjunto de datos no se descubrieron hasta 2010 o después.
Esto se debe en gran parte a la misión *Kepler*, cuyo objetivo era encontrar planetas eclipsantes alrededor de otras estrellas utilizando un telescopio espacial especialmente diseñado.

La siguiente tabla resume algunas otras agregaciones integradas de Pandas:

| Agregación | Devoluciones |
|--------------------|---------------------- -----------|
| ``contar`` | Número total de artículos |
| ``primero``, ``último`` | Primer y último elemento |
| ``media``, ``mediana`` | Media y mediana |
| ``mín``, ``máx`` | Mínimo y máximo |
| ``std``, ``var`` | Desviación estándar y varianza |
| ``loco`` | Desviación absoluta media |
| ``prod`` | Producto de todos los artículos |
| ``suma`` | Suma de todos los artículos |

Todos estos son métodos de objetos `DataFrame` y `Series`.

Sin embargo, para profundizar en los datos, los agregados simples a menudo no son suficientes.
El siguiente nivel de resumen de datos es la operación "groupby", que le permite calcular agregaciones de forma rápida y eficiente en subconjuntos de datos.

## groupby: dividir, aplicar, combinar

Las agregaciones simples pueden darle una idea de su conjunto de datos, pero a menudo preferimos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en la llamada operación "groupby".
El nombre "agrupar por" proviene de un comando en el lenguaje de base de datos SQL, pero quizás sea más ilustrativo pensar en los términos acuñados por primera vez por Hadley Wickham de Rstats: *dividir, aplicar, combinar*.

### Dividir, Aplicar, Combinar

En esta figura se ilustra un ejemplo canónico de esta operación de división, aplicación y combinación, donde "aplicar" es una agregación de suma:

![](imagenes/03.08-split-apply-combine.png)

([fuente de la figura en el Apéndice](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/06.00-Figure-Code.ipynb#Split-Apply-Combine))

Esto ilustra lo que logra la operación "groupby":

- El paso *dividir* implica dividir y agrupar un `DataFrame` dependiendo del valor de la clave especificada.
- El paso *aplicar* implica calcular alguna función, generalmente un agregado, una transformación o un filtrado, dentro de los grupos individuales.
- El paso *combinar* fusiona los resultados de estas operaciones en una matriz de salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y fusión cubiertos anteriormente, una comprensión importante es que *no es necesario crear instancias explícitas de las divisiones intermedias*. Más bien, "groupby" puede (a menudo) hacer esto en una sola pasada sobre los datos, actualizando la suma, la media, el recuento, el mínimo u otro agregado para cada grupo a lo largo del camino.
El poder de "groupby" es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo bajo el capó, sino que puede pensar en la *operación como un todo*.

Como ejemplo concreto, veamos el uso de Pandas para el cálculo que se muestra en la siguiente figura.
Comenzaremos creando la entrada `DataFrame`:

In [11]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de dividir, aplicar y combinar se puede calcular con el método `groupby` del `DataFrame`, pasando el nombre de la columna clave deseada:

In [12]:
df.groupby('key')

Observe que lo que se devuelve es un objeto `DataFrameGroupBy`, no un conjunto de objetos `DataFrame`.
Este objeto es donde está la magia: puede considerarlo como una vista especial del "DataFrame", que está preparado para profundizar en los grupos pero no realiza ningún cálculo real hasta que se aplica la agregación.
Este enfoque de "evaluación diferida" significa que los agregados comunes se pueden implementar de manera eficiente de una manera casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto `DataFrameGroupBy`, que realizará los pasos apropiados de aplicación/combinación para producir el resultado deseado:

In [13]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


El método `suma` es sólo una posibilidad aquí; puede aplicar la mayoría de las funciones de agregación de Pandas o NumPy, así como la mayoría de las operaciones de `DataFrame`, como verá en la siguiente discusión.

### El objeto GroupBy

El objeto ``GroupBy` es una abstracción flexible: en muchos sentidos, puede tratarse simplemente como una colección de ``DataFrame``s, aunque hace cosas más sofisticadas bajo el capó. Veamos algunos ejemplos usando los datos de Planetas.

Quizás las operaciones más importantes disponibles mediante `GroupBy` sean *agregar*, *filtrar*, *transformar* y *aplicar*.
Discutiremos cada uno de estos más detalladamente en la siguiente sección, pero antes de eso, echemos un vistazo a algunas de las otras funciones que se pueden usar con la operación básica "GroupBy".

#### Indexación de columnas

El objeto `GroupBy` admite la indexación de columnas de la misma manera que `DataFrame` y devuelve un objeto `GroupBy` modificado.
Por ejemplo:

In [14]:
planets.groupby('method')

In [15]:
planets.groupby('method')['orbital_period']

Aquí hemos seleccionado un grupo "Series" particular del grupo "DataFrame" original por referencia a su nombre de columna.
Al igual que con el objeto `GroupBy`, no se realiza ningún cálculo hasta que llamemos a algún agregado en el objeto:

In [16]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método.

#### Iteración sobre grupos

El objeto `GroupBy` admite la iteración directa sobre los grupos, devolviendo cada grupo como una `Serie` o `DataFrame`:

In [17]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Esto puede ser útil para la inspección manual de grupos con el fin de depurar, pero a menudo es mucho más rápido usar la funcionalidad incorporada "aplicar", que discutiremos en un momento.

#### Métodos de envío

A través de algo de magia de clase Python, cualquier método no implementado explícitamente por el objeto `GroupBy` será pasado y llamado en los grupos, ya sean objetos `DataFrame` o `Series`.
Por ejemplo, usar el método `describe` es equivalente a llamar a `describe` en el `DataFrame` que representa cada grupo:

In [18]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

Mirar esta tabla nos ayuda a comprender mejor los datos: por ejemplo, la gran mayoría de los planetas hasta 2014 fueron descubiertos mediante los métodos de velocidad radial y tránsito, aunque este último método se volvió común más recientemente.
Los métodos más nuevos parecen ser la variación del tiempo de tránsito y la modulación del brillo orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Observe que estos métodos de envío se aplican *a cada grupo individual* y los resultados luego se combinan dentro de "GroupBy" y se devuelven.
Nuevamente, cualquier método `DataFrame`/`Series` válido se puede llamar de manera similar en el objeto `GroupBy` correspondiente.

### Agregar, filtrar, transformar, aplicar

La discusión anterior se centró en la agregación para la operación combinada, pero hay más opciones disponibles.
En particular, los objetos "GroupBy" tienen métodos "agregar", "filtrar", "transformar" y "aplicar" que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

Para los fines de las siguientes subsecciones, usaremos este ``DataFrame``:

In [19]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Agregación

Ahora está familiarizado con las agregaciones "GroupBy" con "suma", "mediana" y similares, pero el método "agregado" permite aún más flexibilidad.
Puede tomar una cadena, una función o una lista de las mismas y calcular todos los agregados a la vez.
Aquí hay un ejemplo rápido que combina todo esto:

In [20]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Otro patrón común es pasar un diccionario que asigna nombres de columnas a las operaciones que se aplicarán en esa columna:

In [21]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Filtrado

Una operación de filtrado le permite eliminar datos según las propiedades del grupo.
Por ejemplo, es posible que deseemos mantener todos los grupos en los que la desviación estándar sea mayor que algún valor crítico:

In [22]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()",
        "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

La función de filtro debe devolver un valor booleano que especifique si el grupo pasa el filtrado. Aquí, debido a que el grupo A no tiene una desviación estándar mayor que 4, se elimina del resultado.

#### Transformación

Si bien la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinarlos.
Para tal transformación, la salida tiene la misma forma que la entrada.
Un ejemplo común es centrar los datos restando la media grupal:

In [23]:
def center(x):
    return x - x.mean()
df.groupby('key').transform(center)

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### El método de aplicación

El método "apply" le permite aplicar una función arbitraria a los resultados del grupo.
La función debe tomar un `DataFrame` y devolver un objeto Pandas (por ejemplo, `DataFrame`, `Series`) o un escalar; el comportamiento del paso de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una operación "aplicar" que normaliza la primera columna por la suma de la segunda:

In [24]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


`apply` dentro de `GroupBy` es flexible: el único criterio es que la función tome un `DataFrame` y devuelva un objeto o escalar Pandas. ¡Lo que hagas en el medio depende de ti!

### Especificación de la clave dividida

En los ejemplos simples presentados anteriormente, dividimos el "DataFrame" en un solo nombre de columna.
Esta es sólo una de las muchas opciones mediante las cuales se pueden definir los grupos, y aquí veremos algunas otras opciones para la especificación de grupos.

#### Una lista, matriz, serie o índice que proporciona las claves de agrupación

La clave puede ser cualquier serie o lista con una longitud que coincida con la del `DataFrame`. Por ejemplo:

In [25]:
L = [0, 1, 0, 1, 2, 0]
df.groupby(L).sum()

,data1,data2
0,7,17
1,4,3
2,4,7


Por supuesto, esto significa que hay otra forma más detallada de lograr el `df.groupby('key')` de antes:

In [26]:
df.groupby(df['key']).sum()

,data1,data2
key,,
A,3,8
B,5,7
C,7,12


#### Un índice de mapeo de diccionario o serie a grupo

Otro método consiste en proporcionar un diccionario que asigne valores de índice a las claves del grupo:

In [27]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
key,,
consonant,12,19


#### Cualquier función de Python

De manera similar al mapeo, puede pasar cualquier función de Python que ingrese el valor del índice y genere el grupo:

In [28]:
df2.groupby(str.lower).mean()

,data1,data2
key,,
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


#### Una lista de claves válidas

Además, cualquiera de las opciones clave anteriores se puede combinar para agruparlas en un índice múltiple:

In [29]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
key,key,,
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Ejemplo de agrupación

Como ejemplo de esto, en unas pocas líneas de código Python podemos juntar todo esto y contar los planetas descubiertos por método y por década:

In [30]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


Esto muestra el poder de combinar muchas de las operaciones que hemos discutido hasta este punto cuando analizamos conjuntos de datos realistas: rápidamente obtenemos una comprensión aproximada de cuándo y cómo se detectaron planetas extrasolares en los años posteriores al primer descubrimiento.

Sugeriría profundizar en estas pocas líneas de código y evaluar los pasos individuales para asegurarse de comprender exactamente qué están haciendo con el resultado.
Sin duda, es un ejemplo algo complicado, pero comprender estos elementos le brindará los medios para explorar sus propios datos de manera similar.